In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Loaded environment variables from .env file.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "cais/Zephyr_RMU"
model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "HuggingFaceH4/zephyr-7b-beta"


# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_type = Datasets.RANDOM_BD
dataset_config = DATASETS_DICT[dataset_type]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

retain_train = local_load_dataset(data_dir, dataset_config["retain_files"])
retain_val = local_load_dataset(data_dir, dataset_config["val_retain_files"])

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]

max_length = 512

splits = [0, 2, 4, 5]
# A, B, C
forget_trains = [
    local_load_dataset(data_dir, unlearn_files[splits[i] : splits[i + 1]])
    for i in range(len(splits) - 1)
]
forget_vals = [
    local_load_dataset(data_dir, val_unlearn_files[splits[i] : splits[i + 1]])
    for i in range(len(splits) - 1)
]

forget_train_records = [
    process_corpus(forget_train, tokenizer, max_length)
    for forget_train in forget_trains
]
forget_val_records = [
    process_mcq(forget_val, tokenizer, max_length) for forget_val in forget_vals
]
forget_train_mcq_records = [
    process_mcq(forget_val, tokenizer, max_length, expand_choices=False)
    for forget_val in forget_vals
]

retain_train_records = process_corpus(retain_train, tokenizer, max_length)
retain_val_records = process_mcq(retain_val, tokenizer, max_length)

Filter:   0%|          | 0/942 [00:00<?, ? examples/s]

Map:   0%|          | 0/942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/942 [00:00<?, ? examples/s]

Map:   0%|          | 0/942 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
eval_dict = {
    chr(ord("A") + i): forget_val_records[i] for i in range(len(forget_train_records))
}
eval_dict["retain"] = retain_val_records

In [5]:
assert dataset_type == Datasets.RANDOM_BD
import os
from relearn.unlearn.gd import train_gd

all_train_records = []
for forget_train_record in forget_train_records:
    all_train_records.extend(forget_train_record)

all_train_mcq_records = []
for forget_train_mcq_record in forget_train_mcq_records:
    all_train_mcq_records.extend(forget_train_mcq_record)

# TODO(tcqian): should this be mcq or not
# i.e. do i learn using corpus text, is this possible

RETRAIN_MODEL = False
path = Path("../models/random_bd") / "init"
if RETRAIN_MODEL:
    model = train_gd(
        model,
        10,
        {"all": all_train_mcq_records},
        {"retain": retain_train_records},
        eval_dict,
        # forget_alphas={"all": 1.0},
        do_forget=False,
        batch_size=4,
        lr=1e-6,
        eval_at_start=False,
    )

    os.makedirs(path, exist_ok=True)

    model.save_pretrained(path)
else:
    model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16).to(
        device
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
import wandb

wandb.login()

config = {
    "model_id": model_id,
    "magnitude": 6.5,
    "lr": 1e-5,
    "n_epochs": 12,
    "forget_alphas": {"A": 0.39422},
    "retain_alphas": {"B": 13.51609, "retain": 1},
    "datasets_config": dataset_config,
}


run = wandb.init(
    project="relearn", config=config, tags=["abc", "debug"], entity="12tqian"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 12tqian. Use `wandb login --relogin` to force relogin


In [7]:
from relearn.unlearn.rmu import train_rmu

# i retain using corpus because everything in unlearning should be corpus, no mcq that doesn't make sense
# mcq should only be used to eval
# i care if the model can speak english, not if it can answer mcq
# force norm to be big so it's random things for weird stuff

model = train_rmu(
    model,
    {
        "A": forget_train_records[0],
    },
    {
        "B": forget_train_records[1],
        "retain": retain_train_records,
    },
    eval_records_dict=eval_dict,
    n_epochs=5,
    magnitude=6.5,
    lr=1e-5,
    forget_alphas={"A": 0.39422},
    retain_alphas={"B": 13.51609, "retain": 1},
    eval_at_start=True,
    max_batches=None,
    verbose=True,
    debug=False,
    tokenizer=tokenizer,
)

  0%|                                                                                                                                                                   | 0/236 [00:00<?, ?it/s]/mnt/align1_drive/tcqian/unlearning_order/src/relearn/unlearn/rmu/unlearn.py:253: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  forget_loss = torch.nn.functional.mse_loss(
100%|██████████████████████████████████████████████████████████████████████████████▋| 235/236 [02:23<00:00,  1.63it/s, A/forget_loss=0.0186, B/retain_loss=0.000119, retain/retain_loss=1.85e-7]/mnt/align1_drive/tcqian/unlearning_order/src/relearn/unlearn/rmu/unlearn.py:253: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([2, 512, 4096])). This will likely lead to incorrect results due to b

In [8]:
path = Path("../models/random_bd") / "unlearn_A_retain_B"
os.makedirs(path, exist_ok=True)
model.save_pretrained(path)

[2024-12-30 19:50:06,417] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/mnt/align1_dri

In [7]:
# forget B
model = train_gd(
    model,
    6,
    {"B": forget_train_records[1]},
    {
        "retain": retain_train_records,
    },
    eval_dict,
    do_forget=True,
    batch_size=4,
    lr=1e-6,
    eval_at_start=False,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:05<00:00, 27.69it/s]


Epoch 0 forget_0 Accuracy: 0.4299363057324841


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:05<00:00, 28.11it/s]


Epoch 0 forget_1 Accuracy: 0.5063694267515924


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:02<00:00, 27.96it/s]


...

Epoch 5 forget_1 Accuracy: 0.47770700636942676


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:02<00:00, 28.13it/s]


Epoch 5 forget_2 Accuracy: 0.5031847133757962


 25%|████████████████████████████▊                                                                                      | 236/942 [04:23<13:07,  1.11s/it, forget_B=0.00299, retain_retain=1.57]

Epoch 5 retain Accuracy: 0.578343949044586


In [ ]:
from relearn.attacks.rtt import train_ft

# relearm only A
model = train_ft(
    model,
    10,
    forget_train_mcq_records[0],
    eval_dict,
    batch_size=4,
    lr=1e-6,
    eval_at_start=False,
)